# 📚 Módulo 1: Fundamentos de RAG
## De Cero a tu Primer Sistema RAG Funcional (75 minutos)

---

### 🎯 Objetivos de este módulo:
1. **Entender** qué es RAG y por qué es revolucionario
2. **Construir** tu primer pipeline RAG desde cero
3. **Experimentar** con chunking, embeddings y retrieval
4. **Medir** latencia, costo y calidad

### ⏱️ Timeline:
- 08:15-08:35: Teoría y conceptos (20 min)
- 08:35-08:55: Implementación guiada (20 min)
- 08:55-09:30: Práctica y experimentación (35 min)

## Parte 1: Setup y Conceptos [08:15-08:35]

### 🧠 ¿Qué es RAG?

**Retrieval-Augmented Generation** = Recuperar + Aumentar + Generar

Es como darle a un LLM:
- 📚 Una biblioteca personal
- 🔍 Un buscador ultra-rápido
- 🎯 Contexto específico para cada pregunta

In [ ]:
# Celda 1: Imports y configuración
import sys
import os
from pathlib import Path
import time
from typing import List, Dict, Any

# Añadir src al path
sys.path.append(str(Path.cwd().parent / 'src'))

# Imports de nuestros módulos
from shared_config import RAGMasterConfig, TestSuite, MetricsTracker, Module

# Configuración
from dotenv import load_dotenv
load_dotenv()

# Verificar API key
api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    print("❌ Por favor configura OPENAI_API_KEY en .env")
else:
    print(f"✅ API Key configurada: {api_key[:7]}...")

# Inicializar tracker de métricas
metrics = MetricsTracker()
config = RAGMasterConfig()

print("\n🚀 Ambiente listo para Módulo 1: Fundamentos")

### 📊 Los 4 Pilares de RAG

In [ ]:
# Celda 2: Visualizar arquitectura RAG
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib.patches import FancyBboxPatch, FancyArrowPatch
import numpy as np

fig, ax = plt.subplots(figsize=(12, 8))

# Componentes
components = [
    {"name": "📄 Documentos\nOriginales", "pos": (1, 6), "color": "lightblue"},
    {"name": "✂️ Text\nSplitter", "pos": (3, 6), "color": "lightgreen"},
    {"name": "🔢 Embeddings\nModel", "pos": (5, 6), "color": "lightyellow"},
    {"name": "💾 Vector\nDatabase", "pos": (7, 6), "color": "lightcoral"},
    {"name": "❓ User\nQuery", "pos": (1, 3), "color": "lightgray"},
    {"name": "🔍 Semantic\nSearch", "pos": (4, 3), "color": "lightgreen"},
    {"name": "📑 Retrieved\nContext", "pos": (7, 3), "color": "lightyellow"},
    {"name": "🤖 LLM\n(GPT)", "pos": (4, 0.5), "color": "lightblue"},
    {"name": "💬 Final\nAnswer", "pos": (7, 0.5), "color": "lightgreen"}
]

# Dibujar componentes
for comp in components:
    box = FancyBboxPatch(
        (comp["pos"][0]-0.4, comp["pos"][1]-0.3),
        0.8, 0.6,
        boxstyle="round,pad=0.1",
        facecolor=comp["color"],
        edgecolor="black",
        linewidth=2
    )
    ax.add_patch(box)
    ax.text(comp["pos"][0], comp["pos"][1], comp["name"], 
            ha="center", va="center", fontsize=10, fontweight="bold")

# Flechas
arrows = [
    ((1.4, 6), (2.6, 6), "Cargar"),
    ((3.4, 6), (4.6, 6), "Chunks"),
    ((5.4, 6), (6.6, 6), "Vectors"),
    ((7, 5.7), (7, 3.3), "Store"),
    ((1.4, 3), (3.6, 3), "Embed"),
    ((4.4, 3), (6.6, 3), "Top-K"),
    ((7, 2.7), (4.4, 0.8), "Context"),
    ((1, 2.7), (3.6, 0.8), "Query"),
    ((4.4, 0.5), (6.6, 0.5), "Generate")
]

for start, end, label in arrows:
    arrow = FancyArrowPatch(
        start, end,
        arrowstyle="->",
        connectionstyle="arc3,rad=0.1",
        linewidth=2,
        color="darkblue"
    )
    ax.add_patch(arrow)
    
    # Label en la flecha
    mid_x = (start[0] + end[0]) / 2
    mid_y = (start[1] + end[1]) / 2
    ax.text(mid_x, mid_y + 0.2, label, fontsize=8, ha="center", style="italic")

# Configuración del plot
ax.set_xlim(0, 8)
ax.set_ylim(-0.5, 7)
ax.set_aspect('equal')
ax.axis('off')

# Título y fases
ax.text(4, 7.5, "🏗️ ARQUITECTURA RAG COMPLETA", fontsize=16, fontweight="bold", ha="center")
ax.text(4, 6.8, "FASE 1: INDEXACIÓN", fontsize=10, color="blue", ha="center")
ax.text(4, 3.8, "FASE 2: RETRIEVAL", fontsize=10, color="green", ha="center")
ax.text(4, 1.3, "FASE 3: GENERACIÓN", fontsize=10, color="red", ha="center")

plt.tight_layout()
plt.show()

print("\n🎯 Los 3 momentos clave de RAG:")
print("1️⃣ INDEXACIÓN: Preparar y almacenar el conocimiento")
print("2️⃣ RETRIEVAL: Encontrar información relevante")
print("3️⃣ GENERACIÓN: Crear respuesta con contexto")

## Parte 2: Implementación Básica [08:35-08:55]

### 🛠️ Construyendo tu Primer RAG

In [ ]:
# Celda 3: Implementación del RAG más simple posible
from openai import OpenAI
import chromadb
import PyPDF2
import numpy as np

class SimpleRAG:
    """Tu primer RAG en 50 líneas de código"""
    
    def __init__(self):
        print("🚀 Inicializando SimpleRAG...")
        self.client = OpenAI(api_key=api_key)
        self.chroma = chromadb.Client()
        
        # Crear colección (como una tabla en base de datos)
        try:
            self.collection = self.chroma.create_collection("simple_rag")
            print("✅ Colección creada")
        except:
            self.chroma.delete_collection("simple_rag")
            self.collection = self.chroma.create_collection("simple_rag")
            print("♻️ Colección recreada")
    
    def load_document(self, filepath: str) -> str:
        """Cargar documento (PDF o TXT)"""
        print(f"📄 Cargando: {filepath}")
        
        if filepath.endswith('.pdf'):
            with open(filepath, 'rb') as file:
                pdf = PyPDF2.PdfReader(file)
                text = ""
                for page in pdf.pages:
                    text += page.extract_text()
        else:
            with open(filepath, 'r', encoding='utf-8') as file:
                text = file.read()
        
        print(f"✅ Documento cargado: {len(text)} caracteres")
        return text
    
    def create_chunks(self, text: str, chunk_size: int = 500) -> List[str]:
        """Dividir texto en chunks"""
        chunks = []
        for i in range(0, len(text), chunk_size):
            chunk = text[i:i+chunk_size]
            chunks.append(chunk)
        
        print(f"✂️ Creados {len(chunks)} chunks de ~{chunk_size} caracteres")
        return chunks
    
    def index_chunks(self, chunks: List[str]):
        """Indexar chunks en vector database"""
        print(f"🔢 Indexando {len(chunks)} chunks...")
        
        # Añadir a ChromaDB (él se encarga de los embeddings)
        self.collection.add(
            documents=chunks,
            ids=[f"chunk_{i}" for i in range(len(chunks))]
        )
        
        print("✅ Chunks indexados en ChromaDB")
    
    def query(self, question: str, k: int = 3) -> str:
        """Hacer una pregunta al RAG"""
        print(f"\n❓ Pregunta: {question}")
        
        # 1. Buscar chunks relevantes
        start_time = time.time()
        results = self.collection.query(
            query_texts=[question],
            n_results=k
        )
        retrieval_time = (time.time() - start_time) * 1000
        
        # 2. Preparar contexto
        context = "\n\n".join(results['documents'][0])
        print(f"🔍 Recuperados {k} chunks relevantes en {retrieval_time:.0f}ms")
        
        # 3. Generar respuesta con LLM
        prompt = f"""
        Contexto:
        {context}
        
        Pregunta: {question}
        
        Instrucciones: Responde basándote ÚNICAMENTE en el contexto proporcionado.
        Si no encuentras la información en el contexto, di "No tengo esa información".
        """
        
        start_time = time.time()
        response = self.client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[{"role": "user", "content": prompt}],
            temperature=0.3,
            max_tokens=200
        )
        generation_time = (time.time() - start_time) * 1000
        
        answer = response.choices[0].message.content
        total_time = retrieval_time + generation_time
        
        # Métricas
        print(f"⏱️ Tiempos: Retrieval={retrieval_time:.0f}ms, Generation={generation_time:.0f}ms, Total={total_time:.0f}ms")
        
        # Registrar en métricas globales
        cost = len(prompt) / 1000 * 0.0015 + len(answer) / 1000 * 0.002  # Aproximado
        metrics.log_query(
            module=Module.BASICS,
            query=question,
            response=answer,
            latency=total_time,
            cost=cost,
            tokens=len(prompt.split()) + len(answer.split())
        )
        
        return answer

# Crear instancia
rag = SimpleRAG()
print("\n✅ ¡Tu primer RAG está listo!")

In [ ]:
# Celda 4: Cargar y procesar documento
# Usamos un documento de ejemplo
doc_path = "../data/company_handbook.pdf"

# Verificar que existe
if not Path(doc_path).exists():
    # Crear documento de ejemplo si no existe
    print("📝 Creando documento de ejemplo...")
    !python ../src/utils.py --create-sample-data

# Cargar documento
text = rag.load_document(doc_path)

# Ver preview
print("\n📖 Preview del documento:")
print("-" * 50)
print(text[:500] + "...")
print("-" * 50)

In [ ]:
# Celda 5: Chunking e Indexación
print("🔨 PASO 1: CHUNKING")
print("=" * 50)

# Crear chunks
chunks = rag.create_chunks(text, chunk_size=500)

# Mostrar estadísticas
print(f"\n📊 Estadísticas de chunking:")
print(f"- Total chunks: {len(chunks)}")
print(f"- Tamaño promedio: {np.mean([len(c) for c in chunks]):.0f} chars")
print(f"- Chunk más pequeño: {min([len(c) for c in chunks])} chars")
print(f"- Chunk más grande: {max([len(c) for c in chunks])} chars")

# Ver algunos chunks
print("\n👀 Ejemplo de chunks:")
for i in range(min(3, len(chunks))):
    print(f"\nChunk {i+1}:")
    print("-" * 40)
    print(chunks[i][:150] + "...")

In [ ]:
# Celda 6: Indexar en Vector Database
print("💾 PASO 2: INDEXACIÓN")
print("=" * 50)

# Indexar solo los primeros 20 chunks para rapidez
# En producción indexarías todos
chunks_to_index = chunks[:20]

print(f"\n🎯 Indexando {len(chunks_to_index)} chunks en ChromaDB...")
start_time = time.time()

rag.index_chunks(chunks_to_index)

indexing_time = (time.time() - start_time) * 1000
print(f"✅ Indexación completada en {indexing_time:.0f}ms")
print(f"⚡ Velocidad: {len(chunks_to_index) / (indexing_time/1000):.1f} chunks/segundo")

## Parte 3: Práctica - Tu Turno [08:55-09:30]

### 🎯 Hora de Experimentar

In [ ]:
# Celda 7: Tu primera query RAG
print("🚀 TU PRIMERA QUERY RAG")
print("=" * 50)

# La query héroe del día
respuesta = rag.query("¿Cuál es la política de vacaciones?")

print("\n💬 Respuesta:")
print("-" * 40)
print(respuesta)
print("-" * 40)

# Verificar si es correcta
evaluation = TestSuite.evaluate_response(respuesta, Module.BASICS)
print(f"\n📊 Evaluación:")
print(f"- Score: {evaluation['score']:.2f}/1.0")
print(f"- ¿Pasó?: {'✅ Sí' if evaluation['passed'] else '❌ No'}")
print(f"- Tiene info básica: {'✅' if evaluation['has_basic_info'] else '❌'}")

In [ ]:
# Celda 8: Experimentar con diferentes queries
print("🧪 EXPERIMENTO: Diferentes tipos de preguntas")
print("=" * 50)

test_queries = [
    "¿Cuál es el horario de trabajo?",
    "¿Hay trabajo remoto?",
    "¿Cuáles son los beneficios?",
    "¿Cómo es el proceso de onboarding?",
    "¿Qué pasa si me caso?"  # Edge case
]

for i, query in enumerate(test_queries, 1):
    print(f"\n{'='*50}")
    print(f"Query {i}: {query}")
    print("-" * 50)
    
    answer = rag.query(query, k=3)
    
    print(f"\n💬 Respuesta:")
    print(answer[:200] + "..." if len(answer) > 200 else answer)
    
    # Pausa para no saturar la API
    time.sleep(1)

In [ ]:
# Celda 9: Experimentar con parámetros
print("⚙️ EXPERIMENTO: Ajustando parámetros")
print("=" * 50)

query_test = "¿Cuáles son todos los beneficios de la empresa?"

# Probar diferentes valores de K (chunks recuperados)
k_values = [1, 3, 5, 10]
results = []

for k in k_values:
    print(f"\n🔍 Probando con k={k} chunks...")
    
    start = time.time()
    answer = rag.query(query_test, k=k)
    latency = (time.time() - start) * 1000
    
    results.append({
        'k': k,
        'answer_length': len(answer),
        'latency_ms': latency,
        'answer_preview': answer[:100]
    })
    
    print(f"- Longitud respuesta: {len(answer)} chars")
    print(f"- Latencia: {latency:.0f}ms")
    time.sleep(1)

# Visualizar resultados
import pandas as pd
df = pd.DataFrame(results)
print("\n📊 Resumen de resultados:")
print(df[['k', 'answer_length', 'latency_ms']])

# Conclusión
print("\n💡 Observación: Más chunks = respuestas más completas pero mayor latencia")

In [ ]:
# Celda 10: Probar con pregunta que NO está en el documento
print("❌ EXPERIMENTO: Pregunta sin respuesta")
print("=" * 50)

# Pregunta que no debería estar en el manual
pregunta_imposible = "¿Cuál es el precio de las acciones de la empresa en bolsa?"

print(f"\n❓ Pregunta: {pregunta_imposible}")
respuesta = rag.query(pregunta_imposible)

print(f"\n💬 Respuesta:")
print("-" * 40)
print(respuesta)
print("-" * 40)

# Verificar si el sistema reconoce que no tiene la información
if "no tengo" in respuesta.lower() or "no encuentro" in respuesta.lower():
    print("\n✅ ¡Bien! El RAG reconoce cuando no tiene información")
else:
    print("\n⚠️ Cuidado: El RAG podría estar alucinando")

## 📊 Análisis de Métricas del Módulo 1

In [ ]:
# Celda 11: Ver métricas acumuladas
print("📈 MÉTRICAS DEL MÓDULO 1")
print("=" * 50)

# Obtener resumen
summary = metrics.get_summary()

if Module.BASICS.name in summary:
    stats = summary[Module.BASICS.name]
    
    print(f"\n📊 Estadísticas:")
    print(f"- Queries realizadas: {stats['queries_count']}")
    print(f"- Latencia promedio: {stats['avg_latency_ms']:.0f}ms")
    print(f"- Costo total: ${stats['total_cost_usd']:.4f}")
    print(f"- Tokens totales: {stats['total_tokens']}")
    
    # Comparar con target
    target = config.target_metrics[Module.BASICS]
    print(f"\n🎯 Comparación con objetivos:")
    print(f"- Latencia: {stats['avg_latency_ms']:.0f}ms vs objetivo {target['latency']}ms")
    print(f"- Costo promedio: ${stats['total_cost_usd']/stats['queries_count']:.4f} vs objetivo ${target['cost']}")
    
    # Visualizar
    metrics.plot_progress()
else:
    print("No hay métricas registradas aún")

## 🎯 Desafíos Adicionales (Si terminas antes)

In [ ]:
# DESAFÍO 1: Mejorar el chunking
def smart_chunking(text: str, chunk_size: int = 500) -> List[str]:
    """
    TODO: Implementa un chunking más inteligente que:
    - No corte frases a la mitad
    - Respete párrafos cuando sea posible
    - Mantenga contexto entre chunks
    """
    # Tu código aquí
    pass

# DESAFÍO 2: Añadir metadatos
def index_with_metadata(chunks: List[str], metadata: List[Dict]):
    """
    TODO: Indexa chunks con metadata adicional como:
    - Número de página
    - Sección del documento
    - Fecha de actualización
    """
    # Tu código aquí
    pass

# DESAFÍO 3: Implementar re-ranking
def rerank_results(query: str, results: List[str]) -> List[str]:
    """
    TODO: Re-ordenar resultados basándose en:
    - Relevancia semántica
    - Longitud del chunk
    - Presencia de keywords
    """
    # Tu código aquí
    pass

print("💪 ¡Intenta resolver estos desafíos!")
print("Pista: Mira el módulo 2 para inspiración 😉")

## 🎉 ¡Felicitaciones!

### ✅ Lo que has logrado en el Módulo 1:

1. **Construiste** tu primer sistema RAG funcional
2. **Indexaste** documentos en un vector database
3. **Realizaste** búsqueda semántica
4. **Generaste** respuestas con contexto
5. **Mediste** latencia y costos

### 📊 Tus métricas actuales:
- ⏱️ Latencia: ~2000ms
- 💰 Costo: ~$0.01 por query
- 🎯 Accuracy: ~70%

### 🚀 En el Módulo 2 mejoraremos:
- Reducir latencia a 1000ms (-50%)
- Reducir costos a $0.008 (-20%)
- Aumentar accuracy a 80% (+10%)

---

**☕ Toma un break de 15 minutos y nos vemos en el Módulo 2!**